# 라이브러리

In [2]:
import pandas as pd
import os
import tqdm
import numpy as np
from datetime import datetime

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [4]:
raw_fold = 'D:/data/big2/BD1/raw'

# Load

## result 최종-수정(전체)

In [5]:
# about 
name = '4등급 차량 리스트 양식(전체) result 최종 Grade 추가(2023.04.13_PM.01.23.26)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), encoding='cp949', low_memory=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 33 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   차대번호      1502235 non-null  object 
 1   차량번호      1502235 non-null  object 
 2   제원관리번호    1502235 non-null  object 
 3   차종        1502235 non-null  object 
 4   용도        1502235 non-null  object 
 5   최초등록일     1502235 non-null  int64  
 6   연식        1502235 non-null  int64  
 7   제작일자      1335178 non-null  float64
 8   검사유효일     1502223 non-null  float64
 9   배출가스인증번호  1382222 non-null  object 
 10  등급        1502235 non-null  int64  
 11  차명        1502235 non-null  object 
 12  차종분류      1502235 non-null  object 
 13  차종유형      1502235 non-null  object 
 14  자동차형식     1501958 non-null  object 
 15  제작사명      1492590 non-null  object 
 16  연료        1502235 non-null  object 
 17  엔진형식      1500400 non-null  object 
 18  총중량       1502234 non-null  float64
 19  적재중량      1501728 non

## 한불모터스 회신파일

In [7]:
name = '한불모터스_배출가스인증번호_검토자료_회신(2023.04.17)'
file_name = f'{name}.xlsx'
hb = pd.read_excel(os.path.join(raw_fold, file_name))

In [8]:
hb.shape

(2672, 31)

# 확인

In [9]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [10]:
df.loc[(df['제작사명'] == '한불모터스(주)') & (df['연료'] == '경유')].shape

(2668, 33)

In [11]:
df['배인번호_수정'] = df['배인번호_수정'].astype('str')

In [12]:
df.loc[df['배인번호_수정'].str.contains('HB')].shape

(2672, 33)

In [13]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'EM Std.', 'DPF 유무', '등급.1'],
      dtype='object')

In [14]:
hb['EM Std.'].unique()

array(['EURO 4 ENGINE EMISSION CONTROL',
       'EURO 3 L4 VEHICLE TECHNICAL EMISSION CONTROL',
       'EURO 5 L6 UPPER ENGINE EMISSION CONTROl'], dtype=object)

# 전처리

## 한불모터스 배인번호_수정

In [15]:
df[['배출가스인증번호', '배인번호_수정']].head()

,배출가스인증번호,배인번호_수정
0,4MY-AD-20,4MY-AD-20
1,4MY-AD-20,4MY-AD-20
2,4MY-AD-20,4MY-AD-20
3,4MY-AD-20,4MY-AD-20
4,4MY-AD-20,4MY-AD-20


In [16]:
hb['배인번호_수정'] = hb['배출가스인증번호']

In [17]:
hb['배인번호_수정'].isnull().sum()

0

## 한불모터스 등급_수정

In [18]:
hb['등급.1'].value_counts(dropna=False)

NaN     2671
3.00       1
Name: 등급.1, dtype: int64

In [19]:
hb.loc[hb['등급.1'] == 3, '등급_수정'] = 3

In [20]:
hb['등급_수정'].value_counts(dropna=False)

NaN     2671
3.00       1
Name: 등급_수정, dtype: int64

## 한불모터스 DPF유무_수정

In [21]:
df['DPF유무_수정'].unique()

array(['무', '유', '확인불가', nan], dtype=object)

In [22]:
# WITHOUT PARTICLE FILTER: DPF 없음
hb['DPF 유무'].value_counts(dropna=False)

WITH PARTICLE EMISSION FILTER    2666
WITHOUT PARTICLE FILTER             6
Name: DPF 유무, dtype: int64

In [23]:
hb['DPF 유무'].unique()

array(['WITH PARTICLE EMISSION FILTER', 'WITHOUT PARTICLE FILTER'],
      dtype=object)

In [24]:
hb.loc[hb['DPF 유무'] == 'WITH PARTICLE EMISSION FILTER', 'DPF유무_수정'] = '유'
hb.loc[hb['DPF 유무'] == 'WITHOUT PARTICLE FILTER', 'DPF유무_수정'] = '무'

In [25]:
hb['DPF유무_수정'].value_counts(dropna=False)

유    2666
무       6
Name: DPF유무_수정, dtype: int64

## 한불모터스 컬럼명 정리

In [26]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

In [27]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', 'EM Std.', 'DPF 유무', '등급.1', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

In [28]:
hb.drop(['EM Std.', 'DPF 유무', '등급.1'], axis=1, inplace=True)

In [29]:
hb.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

## 한불모터스 샘플 result와 부분 교체

### df에서 한불모터스 샘플 제거
- 배인번호_수정에 'HB' 포함된 샘플

In [30]:
df.loc[df['배인번호_수정'].str.contains('HB')].shape

(2672, 33)

In [31]:
pop_idx = df.loc[df['배인번호_수정'].str.contains('HB')].index
len(pop_idx)

2672

In [32]:
df.shape

(1502235, 33)

In [33]:
df_pop_hb = df.drop(pop_idx).reset_index(drop=True)
df_pop_hb.shape

(1499563, 33)

### 한불모터스 제거한 df와 한불모터스 회신 파일 병합

In [34]:
total = pd.concat([df_pop_hb, hb], ignore_index=True)
total.shape

(1502235, 33)

In [35]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'fuel', 'Grade'],
      dtype='object')

#### 컬럼 정리

In [36]:
total.drop(['fuel', 'Grade'], axis=1, inplace=True)

In [37]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '배인번호_수정', '등급_수정', 'DPF유무_수정'],
      dtype='object')

#### 출력

In [41]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%H.%M.%S")
today_date

'2023.04.18_AM.09.22.32'

In [39]:
# about 19.6s
name = f'4등급_차량_리스트_양식_result_최종(전체)({today_date})'
file_name = f'{name}.csv'
total.to_csv(os.path.join(raw_fold, file_name), index=False, encoding='cp949')

In [43]:
name = f'4등급_차량_리스트_양식_result_최종(전체)({today_date})'
name

'4등급_차량_리스트_양식_result_최종(전체)(2023.04.18_AM.09.22.32)'

In [40]:
# about 12m 8.3s
total.iloc[:700000].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result_최종(전체)({today_date})01.xlsx'), index=False)
total.iloc[700000:].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result_최종(전체)({today_date})02.xlsx'), index=False)

# 코드 마지막